In [219]:
from dsm import datasets
from dsm.dsm_api import DeepSurvivalMachines
import numpy as np
from sklearn.model_selection import ParameterGrid
from sksurv.metrics import concordance_index_ipcw, brier_score,cumulative_dynamic_auc

In [220]:
x, t, e = datasets.load_dataset('SUPPORT')

In [221]:
times = np.quantile(t[e == 1], [0.25, 0.5, 0.75]).tolist()

In [222]:
times

[14.0, 58.0, 252.0]

In [223]:
print(x.shape)
print(t.shape)
print(e.shape)

(9105, 44)
(9105,)
(9105,)


In [227]:
train_size = round(9105*0.70)
val_size = round(9105*0.10)
test_size = round(9105*0.20)

x_train, t_train, e_train = x[: train_size], t[: train_size], e[:train_size]
x_test,  t_test,  e_test = x[-test_size:], t[-test_size:], e[-test_size:]
x_val,  t_val,  e_val = x[train_size:train_size+val_size], t[train_size:train_size+val_size], e[train_size:train_size+val_size]

In [228]:
x_test.shape

(1821, 44)

In [229]:
x_val.shape

(910, 44)

In [230]:
x_train.shape

(6374, 44)

In [231]:
print(x_train.shape, t_train.shape, e_train.shape)
print(x_val.shape, t_val.shape, e_val.shape)
print(x_test.shape, t_test.shape, e_test.shape)

(6374, 44) (6374,) (6374,)
(910, 44) (910,) (910,)
(1821, 44) (1821,) (1821,)


In [232]:
et_train = np.array([(e_train[i], t_train[i]) for i in range(len(e_train))],
                 dtype=[('e', bool), ('t', int)])
et_test = np.array([(e_test[i], t_test[i]) for i in range(len(e_test))],
                 dtype=[('e', bool), ('t', int)])
et_val = np.array([(e_val[i], t_val[i]) for i in range(len(e_val))],
                 dtype=[('e', bool), ('t', int)])

In [233]:
print(et_train.shape, et_test.shape, et_val.shape)

(6374,) (1821,) (910,)


In [234]:
param_grid = {'k': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
              'distribution': ['LogNormal', 'Weibull']}

In [235]:
grids = ParameterGrid(param_grid)

In [205]:
#scores = []
for grid in grids:
    model = DeepSurvivalMachines(k = grid['k'], distribution = grid['distribution'])
    model.fit(x_train, t_train, e_train)
    out_risk = model.predict_risk(x_val, times)
    out_survival = model.predict_survival(x_val, times)
    
    cis = []
    brs = []
    for i in range(len(times)):
        cis.append(concordance_index_ipcw(et_train, et_val, out_risk[:, i], times[i])[0])
    
    brs.append(brier_score(et_train, et_val, out_survival, times )[1])
    print ("Concordance Index:", np.mean(cis))
    print ("Brier Score:", np.mean(brs))
    cdauc=[]
    for i in range(len(times)):
        cdauc.append(cumulative_dynamic_auc(et_train, et_val, out_risk[:,i], times[i])[0])
    print ("Cumulative_dynamic AUC ", np.mean(cdauc))  
    print(grid)
    #scores  = scores + [[grid, np.mean(cis), np.mean(brs), np.mean(cdauc)]]
    

  0%|          | 44/10000 [00:00<00:22, 437.38it/s]

Concordance Index: 0.5092904206321293
Brier Score: 0.2094134782813962
Cumulative_dynamic AUC  0.5086922832567786
{'distribution': 'LogNormal', 'k': 1}


  1%|          | 59/10000 [00:00<00:16, 589.59it/s]

Concordance Index: 0.6225696126219331
Brier Score: 0.19564178498171694
Cumulative_dynamic AUC  0.6463304993181905
{'distribution': 'LogNormal', 'k': 2}


  0%|          | 39/10000 [00:00<00:25, 386.10it/s]

...

Concordance Index: 0.5361255567804992
Brier Score: 0.20410873790652573
Cumulative_dynamic AUC  0.5468374972837957
{'distribution': 'Weibull', 'k': 8}


  1%|          | 65/10000 [00:00<00:15, 646.81it/s]

Concordance Index: 0.6072673284548974
Brier Score: 0.20075580259939282
Cumulative_dynamic AUC  0.6229017351810217
{'distribution': 'Weibull', 'k': 9}


100%|██████████| 1/1 [00:00<00:00,  4.37it/s]

Concordance Index: 0.6125137904891208
Brier Score: 0.1990688415320855
Cumulative_dynamic AUC  0.6332807149415878
{'distribution': 'Weibull', 'k': 10}


In [236]:
model = DeepSurvivalMachines(k = 10, distribution = 'LogNormal')
model.fit(x_train, t_train, e_train)
out_risk = model.predict_risk(x_test, times)
out_survival = model.predict_survival(x_test, times)
    
cis = []
brs = []
for i in range(len(times)):
    cis.append(concordance_index_ipcw(et_train, et_test, out_risk[:,i], times[i])[0])

brs.append(brier_score(et_train, et_test, out_survival, times)[1])
print ("Concordance Index:", np.mean(cis))
print ("Brier Score:", np.mean(brs))
cdauc=[]
for i in range(len(times)):
    cdauc.append(cumulative_dynamic_auc(et_train, et_test, out_risk[:,i], times[i])[0])
print ("Cumulative_dynamic AUC ", np.mean(cdauc))      

100%|██████████| 1/1 [00:00<00:00,  3.37it/s]

Concordance Index: 0.6581903218379163
Brier Score: 0.18598571651278992
Cumulative_dynamic AUC  0.6829802892087405
